In [2]:
%matplotlib inline
import requests
import pandas as pd
import json
import plotly.express as px

In [3]:
USERNAME = 'TheRealFergus'
SPORT = 'nfl'

In [4]:
def _clean_int(x):
    if x not in ["", "0", None]:
        return int(x) 
    else:
        return None
    
def _clean_height(height):
    if height in ["", "0", None]:
        return None
    try:
        height = int(height)
    except:
        height = height.strip("\"").split('\'')
        height = int(height[0]) * 12 + int(height[1])

    if height > 96 or height < 36:
        return None
    else:
        return height

In [5]:
def _clean_players(players):

    teams = pd.DataFrame({key: players[key] for key in players.keys() if key.isalpha()}).transpose()
    players = pd.DataFrame({key: players[key] for key in players.keys() if key.isnumeric()}).transpose()

    players = (players
               .assign(years_exp = players["years_exp"].apply(_clean_int))
               .assign(weight = players["weight"].apply(_clean_int))
               .assign(age = players["age"].apply(_clean_int))
               .assign(depth_chart_order = players["depth_chart_order"]
                                           .apply(_clean_int))
               .assign(height = players["height"].apply(_clean_height))
               .assign(birth_date = pd.to_datetime(players["birth_date"], 
                                                   format = '%Y-%m-%d')))
    
    return players, teams

In [6]:
def get_player_data(save = True, filename = 'players.json', include_teams = False):
    """Gets all NFL Player data from the Sleeper API. Only call this once per 
    day as requested by Sleeper. The file is ~ 12MB. Automatically saves the 
    data in a json file named 'players.json'

    Args:
        save (bool, optional): If False, a file of the player data will not be 
        saved. Defaults to True.

        filename (str, optional): The filename to save the player data. Defaults
        to 'players.json'.

        include_teams (bool, optional): If Ture, includes returns team defenses
        as a second output

    Returns:
        dict: A dictionary containing all NFL player data. If include_teams is 
        True, also outputs a pandas dataframe containg team defense data.
    """
    response = requests.get(f'https://api.sleeper.app/v1/players/nfl')  

    players = response.json()

    if save:
        with open(filename, 'w') as json_file:
            json.dump(players, json_file)

    players, teams = _clean_players(players)
    
    if not include_teams:
        return players
    else:
        return players, teams  

In [7]:
def load_player_data(filename = 'players.json', include_teams = False):
    """Loads player data from a json file. Default behavior will load data from 
    the file saved by the function get_player_data()

    Args:
        filename (str, optional): The filename to load. Defaults to 
        'players.json'.

    Returns:
        dict: A dictionary containing all NFL player data.
    """
    with open('players.json', 'r') as json_file:
        players = json.load(json_file)

    players, teams = _clean_players(players)
    
    if not include_teams:
        return players
    else:
        return players, teams 

In [8]:
# Gets the user id
user = requests.get(f'https://api.sleeper.app/v1/user/{USERNAME}').json()
user_id = user["user_id"]

# Gets the current league id and league data
season = '2024'
leagues =  requests.get(f'https://api.sleeper.app/v1/user/{user_id}/leagues/{SPORT}/{season}').json()
league_id = leagues[0]['league_id']
league = requests.get(f'https://api.sleeper.app/v1/league/{league_id}').json()

# Saves the scoring settings of the league for use in projected points
settings = league["scoring_settings"]

# Gets all rosters for the league
rosters = requests.get(f'https://api.sleeper.app/v1/league/{league_id}/rosters').json()

# Saves all owner id's in the league
owner_ids = [rosters[i]["owner_id"] for i in range(len(rosters))]

# Gets all the owner's data for the league
owners = []
for owner_id in owner_ids:
    owners.append(requests.get(f'https://api.sleeper.app/v1/user/{owner_id}').json())

# Saves owner usernames in a series indexed by user id
usernames = [owners[i]['username'] for i in range(len(owners))]
users = pd.Series(usernames, index = owner_ids, name = 'username')
users.index.name = "user_id"

# Saves rosters in a dictionary with user id's as keys
rosters = {rosters[i]['owner_id']: rosters[i] for i in range(len(rosters))}

In [275]:
def get_user_data(username):
    #Works for username or user_id
    return requests.get(f'https://api.sleeper.app/v1/user/{username}').json()

def get_league_ids(user_id, year):
    leagues = requests.get(f'https://api.sleeper.app/v1/user/{user_id}/leagues/nfl/{year}').json()
    ids = [league["league_id"] for league in leagues]
    return ids

def get_leage(league_id):
    return requests.get(f'https://api.sleeper.app/v1/league/{league_id}').json()

def get_owners(league_id):

    rosters = requests.get(f'https://api.sleeper.app/v1/league/{league_id}/rosters').json()

    rosters = pd.DataFrame(rosters)

    user_ids = rosters["owner_id"]
    users = [get_user_data(user_id) for user_id in user_ids]
    users = pd.DataFrame(users)

    settings = pd.DataFrame(list((rosters["settings"])))
    rosters = rosters.merge(settings, left_index=True,  right_index=True).drop(columns = "settings")

    owners = (users
              .merge(rosters, left_on = 'user_id', right_on = 'owner_id')
              .dropna(how = 'all', axis=1))
    
    return owners

In [276]:
players = load_player_data()
user = get_user_data(USERNAME)
user_id = user["user_id"]
league_id = get_league_ids(user_id, 2024)[0]
league = get_leage(league_id)
owners = get_owners(league_id)


In [203]:
user_ids = pd.DataFrame(rosters)["owner_id"]
users = [get_user_data(user_id) for user_id in user_ids]

In [272]:
users = pd.DataFrame(users)
users["phone"].isna()

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
Name: phone, dtype: bool

In [239]:
rosters = pd.DataFrame(rosters)


In [242]:
settings = pd.DataFrame(list((rosters["settings"])))

In [252]:
rosters = rosters.merge(settings, left_index=True,  right_index=True).drop(columns = "settings")
rosters.columns

KeyError: "['settings'] not found in axis"

In [253]:
owners = users.merge(rosters, left_on = 'user_id', right_on = 'owner_id')
owners.columns

Index(['verification', 'username', 'user_id', 'token', 'summoner_region',
       'summoner_name', 'solicitable', 'real_name', 'phone', 'pending',
       'notifications', 'metadata_x', 'is_bot', 'email', 'display_name',
       'deleted', 'data_updated', 'currencies', 'created', 'cookies', 'avatar',
       'taxi', 'starters', 'roster_id', 'reserve', 'players', 'player_map',
       'owner_id', 'metadata_y', 'league_id', 'keepers', 'co_owners', 'wins',
       'waiver_position', 'waiver_budget_used', 'total_moves', 'ties',
       'losses', 'fpts', 'division'],
      dtype='object')

In [10]:
players = get_player_data()

# Gets player projections for the year
year = '2024'
projs = requests.get(f'https://api.sleeper.com/projections/nfl/{year}?season_type=regular&position[]=QB&position[]=RB&position[]=TE&position[]=WR&order_by=pts_dynasty_2qb').json()    

# Saves the player ids of the player
projections = [{'player_id': player["player_id"]} for player in projs]

# Adds the players projected stat totals to the projections
for i in range(len(projs)):
    for (stat, project) in projs[i]["stats"].items():
        projections[i][stat] = project

# Reformats the projects as a dataframe
projections = pd.DataFrame(projections).set_index("player_id")

# Defines the columns of the projections dataframe
# Only the columns that are relevant to the league settings are saved
cols = list(set(projections.columns) & set(settings.keys()))
projections = projections[cols]

# Multiplies each stat by their respective point values from the league settings
for (stat, points) in settings.items():
    if stat in cols:
        projections = projections.assign(**{stat : projections[stat] * points})

# Assigns the total projected points to the dataframe
projections = projections.assign(projected = projections.iloc[:,1:].sum(axis = 1))

# Merges the projections with the player data
projections = projections.merge(players, left_index = True, right_index = True)


In [45]:
projections

[{'player_id': '9999'},
 {'player_id': '9998'},
 {'player_id': '9997'},
 {'player_id': '982'},
 {'player_id': '9759'},
 {'player_id': '9758'},
 {'player_id': '9757'},
 {'player_id': '9756'},
 {'player_id': '9754'},
 {'player_id': '9753'},
 {'player_id': '973'},
 {'player_id': '964'},
 {'player_id': '963'},
 {'player_id': '96'},
 {'player_id': '956'},
 {'player_id': '954'},
 {'player_id': '9512'},
 {'player_id': '9511'},
 {'player_id': '9510'},
 {'player_id': '9509'},
 {'player_id': '9508'},
 {'player_id': '9507'},
 {'player_id': '9506'},
 {'player_id': '9505'},
 {'player_id': '9504'},
 {'player_id': '9503'},
 {'player_id': '9502'},
 {'player_id': '9501'},
 {'player_id': '9500'},
 {'player_id': '9499'},
 {'player_id': '9498'},
 {'player_id': '9497'},
 {'player_id': '9496'},
 {'player_id': '9495'},
 {'player_id': '9494'},
 {'player_id': '9493'},
 {'player_id': '9492'},
 {'player_id': '9491'},
 {'player_id': '9490'},
 {'player_id': '9489'},
 {'player_id': '9488'},
 {'player_id': '9487'},


In [59]:
year = '2024'
projs = requests.get(f'https://api.sleeper.com/projections/nfl/{year}?season_type=regular&position[]=QB&position[]=RB&position[]=TE&position[]=WR&order_by=pts_dynasty_2qb').json()    
projections = [{'player_id': player["player_id"]} for player in projs]

for i in range(len(projs)):
    for (stat, project) in projs[i]["stats"].items():
        projections[i][stat] = project

# Reformats the projects as a dataframe
projections = pd.DataFrame(projections).set_index("player_id")
projections.loc['5849'] *

rush_yd                  526.00
rush_td                    5.00
rush_fd                   52.60
rush_att                  94.00
pts_std                  307.28
pts_ppr                  307.28
pts_half_ppr             307.28
pass_yd                 3717.00
pass_td                   23.00
pass_int                  12.00
pass_fd                  371.70
pass_cmp                 339.00
pass_att                 525.00
pass_2pt                   1.00
gp                        18.00
fum_lost                   3.00
cmp_pct                   64.57
adp_std                   79.50
adp_rookie               999.00
adp_ppr                   79.50
adp_idp                   37.40
adp_half_ppr              79.50
adp_dynasty_std           89.00
adp_dynasty_ppr           89.00
adp_dynasty_half_ppr      89.00
adp_dynasty_2qb           23.50
adp_dynasty              999.00
adp_2qb                   20.30
rec_yd                      NaN
rec_td                      NaN
rec_fd                      NaN
rec_5_9 

In [56]:
projections

,rush_yd,rush_td,rush_fd,rush_att,pts_std,pts_ppr,pts_half_ppr,pass_yd,pass_td,pass_int,...,rec_30_39,rec_20_29,rec_10_19,rec_0_4,rec,bonus_rec_wr,bonus_rec_rb,bonus_rec_te,rush_2pt,def_kr_td
player_id,,,,,,,,,,,,,,,,,,,,,
9999,169.0,2.0,16.9,48.0,260.70,260.70,260.70,3845.0,24.0,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9998,26.0,NaN,2.6,5.0,13.28,13.28,13.28,192.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,51.0,NaN,5.1,8.0,132.30,213.30,172.80,NaN,NaN,NaN,...,8.1,16.2,24.3,16.2,81.0,81.0,NaN,NaN,NaN,NaN
982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10214,NaN,NaN,NaN,NaN,19.20,32.20,25.70,NaN,NaN,NaN,...,1.3,2.6,3.9,2.6,13.0,NaN,NaN,13.0,NaN,NaN
10213,35.0,NaN,3.5,6.0,47.80,77.80,62.80,NaN,NaN,NaN,...,3.0,6.0,9.0,6.0,30.0,30.0,NaN,NaN,NaN,NaN
10212,NaN,NaN,NaN,NaN,29.40,51.40,40.40,NaN,NaN,NaN,...,2.2,4.4,6.6,4.4,22.0,NaN,NaN,22.0,NaN,NaN


In [51]:
set(projections.columns) - set(settings.keys())

{'adp_2qb',
 'adp_dynasty',
 'adp_dynasty_2qb',
 'adp_dynasty_half_ppr',
 'adp_dynasty_ppr',
 'adp_dynasty_std',
 'adp_half_ppr',
 'adp_idp',
 'adp_ppr',
 'adp_rookie',
 'adp_std',
 'cmp_pct',
 'def_kr_td',
 'gp',
 'pass_att',
 'pass_fd',
 'pts_half_ppr',
 'pts_ppr',
 'pts_std',
 'rush_att'}

In [57]:
set(settings.keys())

{'blk_kick',
 'bonus_rec_rb',
 'bonus_rec_te',
 'bonus_rec_wr',
 'def_st_ff',
 'def_st_fum_rec',
 'def_st_td',
 'def_td',
 'ff',
 'fgm_0_19',
 'fgm_20_29',
 'fgm_30_39',
 'fgm_40_49',
 'fgm_50p',
 'fgmiss',
 'fum',
 'fum_lost',
 'fum_rec',
 'fum_rec_td',
 'int',
 'pass_2pt',
 'pass_cmp',
 'pass_inc',
 'pass_int',
 'pass_int_td',
 'pass_sack',
 'pass_td',
 'pass_yd',
 'pts_allow_0',
 'pts_allow_14_20',
 'pts_allow_1_6',
 'pts_allow_21_27',
 'pts_allow_28_34',
 'pts_allow_35p',
 'pts_allow_7_13',
 'rec',
 'rec_0_4',
 'rec_10_19',
 'rec_20_29',
 'rec_2pt',
 'rec_30_39',
 'rec_40p',
 'rec_5_9',
 'rec_fd',
 'rec_td',
 'rec_yd',
 'rush_2pt',
 'rush_fd',
 'rush_td',
 'rush_yd',
 'sack',
 'safe',
 'st_ff',
 'st_fum_rec',
 'st_td',
 'xpm',
 'xpmiss'}

In [11]:
# Returns a dataframe containing a user's team and projected points
# The function takes either a user id or user name
def get_team(user_id = None, username = None):
    
    if username == user_id:
        return None
    elif username != None:
        user_id = users[users == username].index[0]

    # Get's a dataframe of player id's from the roster
    team = pd.DataFrame(pd.Series(rosters[user_id]["players"], name = "player_id"))

    # Merges players with projections dataframe, matches up players with owners
    team = (projections
            .merge(team, left_on='player_id', right_on='player_id', how = 'right')
            .sort_values(by = 'projected', ascending = False))
    
    team["owner"] = users[user_id]

    team = team.set_index('player_id')

    return team

In [12]:
# Save projections for all teams
teams = pd.concat([get_team(user_id = user_id) for user_id in users.index])


In [13]:
# Returns the starters with the highest projected points
def get_starters(user_id = None, username = None):

    # Gets the user_id if username was passed
    if username == user_id:
        return None
    elif username != None:
        user_id = users[users == username].index[0]

    # Gets the team
    team = get_team(user_id)

    starters = pd.DataFrame()

    # The remaining positions available
    positions = ['QB', 'WR', 'WR', 'RB', 'RB', 'TE', 'F', 'F', 'F', 'SF']

    # Adds a boolean column that denotes if the player is a Flex
    team = (team
            .sort_values(by = 'projected', ascending=False)
            .assign(is_flex = team["position"].isin(["WR", "RB", "TE"])))

    # Chooses the highest projected players and eliminates spots from positions
    for id in team.index:

        position = team.loc[id,"position"]
        is_flex = team.loc[id,'is_flex']

        if (position in positions) and len(positions) > 0:
            positions.remove(position)
            starters[id] = team.loc[id]

        elif is_flex and 'F' in positions and len(positions) > 0:
            positions.remove('F')
            starters[id] = team.loc[id]
            
        elif 'SF' in positions and len(positions) > 0:
            positions.remove('SF')
            starters[id] = team.loc[id]

    starters = (starters
                .transpose()
                .drop(columns="is_flex")
                )
    starters = starters.assign(projected = starters["projected"].astype(float))

    return starters

In [14]:
# Saves a dataframe of each player's best starting lineup
starters = []
relevant_cols = ["name", "projected", "position", "team", "owner"]

for user_id in users.index:
    starters.append(get_starters(user_id = user_id))

starters = pd.concat(starters)


In [15]:
# Plots the total contribution of each owner's positions to their total projected score
by_position = starters.pivot_table(index = 'owner',
                                  columns = 'position',
                                  values = 'projected',
                                  aggfunc = 'sum')

by_position = (by_position.assign(total = by_position.sum(axis=1))
                          .sort_values(by='total', ascending = False)
                          .reset_index())
color_seq = [2, 4 , 0, 1]
color_seq = [px.colors.qualitative.T10[i] for i in color_seq]
fig = px.bar(by_position,
       x = 'owner', 
       y = ['QB', 'RB', 'WR', 'TE'],
       barmode = 'group',
       color_discrete_sequence = color_seq)

fig.update_layout(width=1200,
                 height=600,
                 xaxis_title = None,
                 yaxis_title = 'Projected Points',
                 title = "2024 Starter Projected Points By Position",
                 title_x = 0.5,
                 font=dict(size=14),
                 margin=dict(t=70, b=75, l=100, r=50),
                 bargap=0.25,
                 legend = dict(
                     title = None,
                     font = dict(size=20),
                     xanchor = "right",
                     y=.98,
                     yanchor = "top",
                     x=0.99
                 ))
fig.show()

In [20]:
# Plots the season total projected points of all players on each team
total_team = (teams.
              groupby("owner")
              [["projected"]]
              .sum()
              .sort_values(by = 'projected')
              .reset_index()
              )

fig = px.bar(total_team,
              y = 'owner',
              x = 'projected',
              orientation = 'h',
              color_discrete_sequence = [px.colors.qualitative.T10[1]])

fig.update_layout(width=900,
                 height=600,
                 xaxis_title = "Total Projected Points",
                 yaxis_title = "",
                 title = '2024 Season Projected Points for All Players',
                 title_x = 0.5,
                 font=dict(size=14),
                 margin=dict(t=70, b=75, l=50, r=50),
                 bargap=0.4)

fig.update_traces(textposition = "inside", texttemplate='%{x:.0f}')

fig.show()

In [21]:
# Plots the season total projected points of the starters on each team
best_starts = (starters
               .groupby("owner")
               [["projected"]]
               .sum()
               .sort_values(by="projected")
               .reset_index()
               )

fig = px.bar(best_starts,
              y = 'owner',
              x = 'projected',
              orientation = 'h',
              color_discrete_sequence = [px.colors.qualitative.T10[0]])

fig.update_layout(width=900,
                 height=600,
                 xaxis_title = "Total Projected Points",
                 yaxis_title = "",
                 title = '2024 Season Projected Points for Top Starters',
                 title_x = 0.5,
                 font=dict(size=14),
                 margin=dict(t=70, b=75, l=50, r=50),
                 bargap=0.4)

fig.update_traces(textposition = "inside", texttemplate='%{x:.0f}')

fig.show()

In [22]:
# Plots the total and starter projected points in a grouped bar chart
totals = (teams.
          groupby("owner")
          [["projected"]]
          .sum()
          .sort_values(by = 'projected')
          .rename(columns = {'projected': 'All Players'})
          )
starts = (starters
          .groupby("owner")
          [["projected"]]
          .sum()
          .sort_values(by="projected")
          .rename(columns = {'projected': 'Top Starters'})
        )

groups = starts.merge(totals, left_index=True, right_index=True).reset_index()

fig = px.bar(groups,
             y = 'owner', 
             x = ["Top Starters", "All Players"], 
             barmode = 'group', 
             orientation = 'h',
             color_discrete_sequence = px.colors.qualitative.T10)

fig.update_layout(width=1200,
                 height=800,
                 xaxis_title = "Projected Points",
                 yaxis_title = "",
                 title = '2024 Season Projected Points',
                 title_x = 0.5,
                 font=dict(size=14),
                 margin=dict(t=70, b=75, l=50, r=50),
                 bargap=0.25,
                 legend = dict(
                     title = None,
                     font = dict(size=20),
                     xanchor = "right",
                     y=0.05,
                     yanchor = "bottom",
                     x=0.95
                 ))

fig.update_traces(textposition = "inside", texttemplate='%{x:.0f}')

fig.show()

In [36]:
relevant_cols = ["full_name", "projected", "position", "team", "owner"]

my_team = get_team(username='therealfergus')[relevant_cols]
alec_team = get_team(username='alecwilson')[relevant_cols]
pat_team = get_team(username ='burgertownthicnred')[relevant_cols]

my_starters = get_starters(username='therealfergus')[relevant_cols]
alec_starters = get_starters(username='alecwilson')[relevant_cols]
pat_starters = get_starters(username ='burgertownthicnred')[relevant_cols]

KeyError: 'projected'

In [37]:
my_team

,full_name,projected,position,team,owner
player_id,,,,,
5849,Kyler Murray,401.48,QB,ARI,therealfergus
4017,Deshaun Watson,350.77,QB,CLE,therealfergus
9999,Will Levis,336.95,QB,TEN,therealfergus
5850,Josh Jacobs,262.40,RB,GB,therealfergus
8205,Isiah Pacheco,258.15,RB,KC,therealfergus
6130,Devin Singletary,218.85,RB,NYG,therealfergus
1466,Travis Kelce,206.30,TE,KC,therealfergus
6790,D'Andre Swift,205.60,RB,CHI,therealfergus
8146,Garrett Wilson,197.60,WR,NYJ,therealfergus


In [121]:
pat_starters

,name,projected,position,team,owner
4046,Patrick Mahomes,461.390,QB,KC,burgertownthicnred
96,Aaron Rodgers,351.480,QB,NYJ,burgertownthicnred
6803,Brandon Aiyuk,262.800,WR,SF,burgertownthicnred
4039,Cooper Kupp,253.775,WR,LAR,burgertownthicnred
9756,Jordan Addison,241.575,WR,MIN,burgertownthicnred
4035,Alvin Kamara,241.275,RB,NO,burgertownthicnred
4018,Joe Mixon,240.100,RB,HOU,burgertownthicnred
5022,Dallas Goedert,200.725,TE,PHI,burgertownthicnred
2374,Tyler Lockett,190.350,WR,SEA,burgertownthicnred
1689,Adam Thielen,148.175,WR,CAR,burgertownthicnred


In [120]:
my_starters

,name,projected,position,team,owner
5849,Kyler Murray,401.480,QB,ARI,therealfergus
4017,Deshaun Watson,350.770,QB,CLE,therealfergus
8146,Garrett Wilson,310.825,WR,NYJ,therealfergus
1466,Travis Kelce,289.100,TE,KC,therealfergus
5850,Josh Jacobs,284.200,RB,GB,therealfergus
8205,Isiah Pacheco,283.325,RB,KC,therealfergus
2216,Mike Evans,266.600,WR,TB,therealfergus
6130,Devin Singletary,239.950,RB,NYG,therealfergus
6790,D'Andre Swift,228.075,RB,CHI,therealfergus
2749,Raheem Mostert,202.750,RB,MIA,therealfergus


In [316]:
pacc_team = get_team(username = 'pacc')[relevant_cols]
pacc_team

,name,projected,position,team,owner
player_id,,,,,
4881,Lamar Jackson,464.29,QB,BAL,pacc
6904,Jalen Hurts,449.41,QB,PHI,pacc
421,Matthew Stafford,369.04,QB,LAR,pacc
7543,Travis Etienne,295.00,RB,JAX,pacc
8112,Drake London,282.90,WR,ATL,pacc
8136,Rachaad White,280.75,RB,TB,pacc
5872,Deebo Samuel,275.50,WR,SF,pacc
6801,Tee Higgins,261.90,WR,CIN,pacc
5012,Mark Andrews,261.40,TE,BAL,pacc
